# HR Diagram Interactives

In [ ]:
# This Jupyter Notebook was initially developed by Juan Cabanela over the Summer of 2018 for use in 
# Minnesota State University Moorhead's introductory astronomy lab activities.

from IPython.display import display
import pandas as pd
import numpy as np
import bqplot as bq
import ipywidgets as widgets
import tempNcolor as t2c

In [ ]:
##
## Define some variables affecting all the plots here
##

OverlayColor = ['darkturquoise']
PlotBkgStyle={'fill': 'black'}
PlotLayout={'width': '400px', 'min_height': '400px'}

# Import some data from MSUM Stellar Catalog (which is really a modified version of Hipparcos data catalog)
MSUMdata = pd.read_csv('data/Hipparcos.csv')

# Compute RGB colors for points based on B-V color
hexcolors = t2c.rgb2hex(t2c.bv2rgb(MSUMdata['B-V']))
hexlist = hexcolors.tolist()

# Import data for four star clusters and subset it by cluster
rawdata = pd.read_csv('data/clusterdata.csv')

brightness = rawdata['Brightness']
ColorIndex = rawdata['B-V']
tuc = (rawdata['Cluster'] == '47tuc')
pleiades = (rawdata['Cluster'] == 'pleiades')
hyades = (rawdata['Cluster'] == 'hyades')
m53 = (rawdata['Cluster'] == 'm53')

# Compute RGB colors for stars in star clusters based on B-V color
hexcolors_tuc = t2c.rgb2hex(t2c.bv2rgb(ColorIndex[tuc]))
hexlist_tuc = hexcolors_tuc.tolist()
hexcolors_m45 = t2c.rgb2hex(t2c.bv2rgb(ColorIndex[pleiades]))
hexlist_m45 = hexcolors_m45.tolist()
hexcolors_m53 = t2c.rgb2hex(t2c.bv2rgb(ColorIndex[m53]))
hexlist_m53 = hexcolors_m53.tolist()
hexcolors_hyades = t2c.rgb2hex(t2c.bv2rgb(ColorIndex[hyades]))
hexlist_hyades = hexcolors_hyades.tolist()

# Define Cluster information Pandas DataFrame

cluster_info = pd.DataFrame(columns=['Name', 'LongName', 'Dist', 'FeH', 'minBright', 'maxBright'])
cluster_info['index'] = ['m45', 'hyades', 'm53', '47tuc']
cluster_info['Name'] = ['Pleiades', 'Hyades', 'M53', '47 Tuc']
cluster_info['LongName'] = ['Pleiades (Solar Composition)', 'Hyades (Solar Composition)', 'M53 (Low in Metals)', '47 Tuc (Low in Metals)']
cluster_info['Dist'] = [ 136, 47, 18000, 4000 ]
cluster_info['FeH'] = [ 0, 0, -1.86 , -0.7 ]
cluster_info['maxBright'] = [ 1e-2, 1e-2, 1e-3 , 1e-3 ]
cluster_info['minBright'] = 1e-8*cluster_info['maxBright']
cluster_info = cluster_info.set_index('index')

# Set the initial cluster in each plot
init_cluster = 'm45'

# load the data for the interpolated stellar evolution models
evol_data = pd.read_csv('data/interpolated_evolution.csv')

# Generate a list of colors and add to that Pandas DataFrame
evol_data['hexcolor'] = t2c.rgb2hex(t2c.bv2rgb(evol_data['B-V']))


## Interactive Figure 1: A Comparison of an HR diagram to a Brightness diagram

This interactive figure was built using the MSUM Stellar catalog of about 1000
stars with measured B-V colors and known distances.  Recall that to measure the
B-V color of a star, we have to have measured the brightness (also called
"flux") of the star in the B and V filters.  To turn a brightness measurement
into a luminosity estimate, we also need an estimate of the distance to the
star.

- The plot on the left is the Luminosity versus B-V color, that is, the **HR Diagram** for the MSUM Stellar Catalog.  
- The plot on the right is those same stars, but plotted up based on their observed brightnesses and B-V color, a **Brightness Diagram**.

If you select a region on the HR diagram, you will see the same stars will be
highlighted on the Brightness diagram.   This lets you see that the same stars
can appear on very different regions in these two kinds of plots.  *Why is
that?*

The checkbox and slider on the far right allow you to highlight stars in a
given distance range on the brightness diagram.  Examine the appearance of a
group of stars in a narrow *range* of distances (like 100 pc) on the brightness
diagram.  Select another group of stars at another distance but similar range.

Can you explain why a group of stars close to each other on the HR diagram can
appear at such varied positions on the Brightness Diagram?

In [ ]:
##
## Define functions for first interactive to highlight a given distance range
##
def plot_distance_subset(mindist, maxdist):
    global Dist_bright
    
    dist_subset = MSUMdata[(MSUMdata['distance']>=mindist) & (MSUMdata['distance']<=maxdist)]
    Dist_bright.x = dist_subset['B-V']
    Dist_bright.y = dist_subset['Brightness']
    return 

def plot_distance_visibility(change):
    global Dist_bright
    
    # Turn on distance group indicator
    Dist_bright.visible = dist_toggle.value
    if (Dist_bright.visible):
        scat_bright.default_opacities = [0.9]
        # Deselect any highlighted region when asking for distance range
        Highlight_lum.selected = []
        react_region_selector(change)
        region_selector.brushing = True   # resets selector to be brushing (not sure if this is working)
    else:
        scat_bright.default_opacities = [1]
    return 

# This function will respond to changes in slider by selecting subset of data.
def dist_range_selection(change):
    global min_dist0, max_dist0, dist_range, force_dist_range
    max_range = dist_range
    
    if (change.new[1] != change.old[1]):
        # Top slider changing
        maxdist = dist_selector.value[1]
        
        # Adjust range is that is desired behavior
        if force_dist_range:
            mindist = maxdist - max_range
            if (mindist < max_range):
                mindist = 0
                maxdist = max_range
        else:
            mindist = dist_selector.value[0]
        
    else:
        # Bottom slider changing
        mindist = dist_selector.value[0]
        # Adjust range is that is desired behavior
        if force_dist_range:
            maxdist = mindist + max_range
            if (maxdist > max_dist0):
                mindist = max_dist0-max_range
                maxdist = max_dist0
        else:
            maxdist = dist_selector.value[1]
                
    # move sliders if necessary
    if force_dist_range:
        dist_selector.value= (mindist, maxdist)
    
    plot_distance_subset(mindist, maxdist)
    return

# This function takes the information from the BrushSelector and acts on it
def react_region_selector(change):
    global transparent
    
    if Highlight_lum.selected is not None:
        # Identify the selected stars
        idx_selected = Highlight_lum.selected

        # Assume zero opacity to start
        opacities_hl = list(transparent)
        for idx in idx_selected:
            opacities_hl[idx]=1

        # Turn on the highlighted points (make them opaque instead of transparent)
        Highlight_bright.default_opacities = list(opacities_hl)
        Highlight_lum.default_opacities = list(opacities_hl)
        
        # Disable distance subset if necessary
        if (len(Highlight_lum.selected) > 0):
            dist_toggle.value = False
    else:
        # Make all highlights transparent again
        opacities_hl = list(transparent)
        Highlight_bright.default_opacities = list(opacities_hl)
        Highlight_lum.default_opacities = list(opacities_hl)        

In [ ]:
##
## Define scales, axes, and tooltips, which could be similar between 
## various implementations of the displayed widgets
##

# Scales to transform the data
min_loglum = np.floor(np.min(np.log10(MSUMdata['Lum'])))
max_loglum = np.ceil(np.max(np.log10(MSUMdata['Lum'])))
min_lum = 10**min_loglum
max_lum = 10**max_loglum
min_logbright = np.floor(np.min(np.log10(MSUMdata['Brightness'])))
max_logbright = min_logbright + (max_loglum-min_loglum)  # Set to have same scale
min_bright = 10**min_logbright
max_bright = 10**max_logbright

x_sc_color = bq.LinearScale(min =-1, max=2)
y_sc_bright = bq.LogScale(min = min_bright, max=max_bright)
y_sc_lum = bq.LogScale(min = min_lum, max=max_lum)

# Labels and scales for Axes
ax_x_color = bq.Axis(label='B-V', scale=x_sc_color, tick_format='.1f', num_ticks=7)
ax_y_bright = bq.Axis(label='Brightness (L_sun/pc^2)', scale=y_sc_bright, orientation='vertical')
ax_y_lum = bq.Axis(label='Luminosity (L_sun)', scale=y_sc_lum, orientation='vertical')

# moving the label perpendicular to the axis
ax_y_bright.label_offset = '3.5em'
ax_y_lum.label_offset = '3em'

# Define the distance limits for brightness plot
min_dist0 = 0
max_dist0 = 5000
init_dist = 1000
dist_range = 100
force_dist_range = False   # If true, distance range is forced to dist_range by dynamically changing limits

In [ ]:
##
## Construct the actual plots and widgets for the first interactive
##

# Define a tooltip
def_tt_bright = bq.Tooltip(fields=['x', 'y'], formats=['.2f', '.2e'], labels=['B-V', 'Brightness'])
def_tt_lum = bq.Tooltip(fields=['x', 'y'], formats=['.2f', '.3f'], labels=['B-V', 'Luminosity'])

# Designing the Brightness diagram
opaque = list(1*MSUMdata['B-V']/MSUMdata['B-V'])
scat_bright = bq.Scatter(x=MSUMdata['B-V'], y=MSUMdata['Brightness'], 
                         names=MSUMdata['HIP'], display_names=False, 
                         scales={'x': x_sc_color, 'y': y_sc_bright},
                         marker='circle', colors=hexlist, 
                         default_size=2, default_opacities=opaque,
                         tooltip=def_tt_bright, stroke=None, fill=True)
scat_bright.tooltip = None

# Designing the HR diagram
scat_lum = bq.Scatter(x=MSUMdata['B-V'], y=MSUMdata['Lum'], 
                      names=MSUMdata['HIP'], display_names=False, 
                      scales={'x': x_sc_color, 'y': y_sc_lum},
                      marker='circle', colors=hexlist, 
                      default_size=2, default_opacities=opaque,
                      tooltip=def_tt_lum, stroke=None, fill=True)
scat_lum.tooltip = None

# This is a second copy of luminosity plot but with zero size points, we can then change the size of a point to
# highlight it.
transparent = list(0*MSUMdata['B-V'])
Highlight_lum = bq.Scatter(x=MSUMdata['B-V'], y=MSUMdata['Lum'], names=MSUMdata['HIP'], display_names=False,
                           default_size=20, default_opacities=transparent,
                           scales={'x': x_sc_color, 'y': y_sc_lum}, 
                           marker='circle', colors=OverlayColor, visible = True
                          )
Highlight_bright = bq.Scatter(x=MSUMdata['B-V'], y=MSUMdata['Brightness'], names=MSUMdata['HIP'], display_names=False,
                              default_size=20, default_opacities=transparent,
                              scales={'x': x_sc_color, 'y': y_sc_bright}, 
                              marker='circle', colors=OverlayColor, visible = True
                             )
# Not sure why default_opacities value isn't sticking in above Scatter calls, but this fixes it - Juan 03.01.2021
Highlight_lum.default_opacities = transparent
Highlight_bright.default_opacities = transparent

# Set up a Brush Selector to select regions of the HR Diagram and highlight the 
# same regions of the Brightness Diagram
region_selector = bq.interacts.BrushSelector(x_scale=x_sc_color, y_scale=y_sc_bright, 
                                             marks=[Highlight_lum], color=OverlayColor[0])


# Initial distance ranges to overlay on Brightness diagram
dist_subset = MSUMdata[(MSUMdata['distance']>=init_dist) & (MSUMdata['distance']<=init_dist+dist_range)]
Dist_bright = bq.Scatter(x=dist_subset['B-V'], y=dist_subset['Brightness'], scales={'x': x_sc_color, 'y': y_sc_bright},
                         marker='circle', colors=OverlayColor, default_size=40, stroke=None, fill=False)
Dist_bright.visible = False

# Displaying the data
fig_lum = bq.Figure(axes=[ax_x_color, ax_y_lum], marks=[scat_lum, Highlight_lum], 
                    title='HR Diagram of Sample Stars', 
                    background_style=PlotBkgStyle, layout=PlotLayout, interaction=region_selector)
fig_bright = bq.Figure(axes=[ax_x_color, ax_y_bright], marks=[scat_bright, Dist_bright, Highlight_bright], 
                       title='Brightness of Sample Stars', 
                       background_style=PlotBkgStyle, layout=PlotLayout)

# Add a distance selection slider and toggle switch
dist_toggle = widgets.Checkbox(value=False, description='Highlight Stars in', disabled=False )
dist_selector = widgets.IntRangeSlider(value=[init_dist, init_dist+dist_range], min=min_dist0, max=max_dist0, step=dist_range, 
                                       description='Distance Range', disabled=False,
                                       continuous_update=True, orientation='vertical', 
                                       readout=True, readout_format='04d')
dist_selector.layout.height = '300px'
dist_selector.layout.width = '125px'
dist_toggle.layout.width = '300px'
                                  
dist_control = widgets.VBox([dist_toggle, dist_selector], 
                            layout=widgets.Layout(align_content='center', align_items='center', 
                                          display='flex', 
                                          flex_flow='column', height='400px', max_height='500px', 
                                          max_width='250px', min_height='50px', min_width='150px', 
                                          overflow='hidden', width='150px'))

# Display it all
main_disp = widgets.HBox([fig_lum, fig_bright, dist_control], 
                     layout=widgets.Layout(align_content='center', align_items='center', 
                                           display='flex', 
                                           flex_flow='row', height='500px', max_height='500px', 
                                           max_width='1000px', min_height='400px', min_width='900px', 
                                           overflow='hidden', width='1000px'))

display(main_disp)

# Respond to changes in the distance range selected
region_selector.observe(react_region_selector, names=['brushing'])
dist_selector.observe(dist_range_selection, 'value')
dist_toggle.observe(plot_distance_visibility, 'value')

## Interactive Figure 2:  Distance Estimates via Main Sequence Fitting

The interactive figure below shows the previous HR diagram on the left and the
observed brightness diagrams of several star clusters on the right.

Most stars we observe lie on the Main Sequence of the HR diagram, which means
if we construct a brightness diagram for stars in a star cluster, we can
compare the brightness of the star cluster's main sequece to the KNOWN
luminosity of the main sequence stars to estimate the distance.

This interactive allows *main sequence fitting*" of the distance to a star
cluster in two steps:

1. The HR diagram has blue-green line representing a main sequence on the HR diagram.
Initially this line has not been 'fit' to the main sequence.  Click and
drag the blue-green dots until the blue-green line representing the main
sequence matches the observed main sequence. You are essentially defining the
luminosity of the main sequence for a series of B-V colors.

2. Did you notice the corresponding blue-green line on the brightness diagram
representing the brightness of your main sequence at 10 parcsecs changed as
your made your previous adjustments? Adjusting the distance slider will change
its brightness to correspond to the new distance.  Adjust the distance until
you match the brightness of the main sequence to the observed main sequence for
your selected star cluster.  When you do this, you will have estimate the
distance to the star cluster via *main sequence fitting*.

**Note**: The Pleiades and 47 Tuc data is fairly 'clean' whereas the Hyades and
M53 data contains foreground stars contaminating the data.  This is meant to
allow you to see a little of what real data looks like that astronomers have to
deal with when attempting this method of distance estimation.

In [ ]:
##
## Define functions for second interactive to allow editable MS on HR and brightness diagrams
##

# Define function to generate brightness curve for a given distance
def generateMScurve(dist):
    global MSscat
    return MSscat.y/(4*np.pi*dist*dist)

# Define function to process distance change
def MSdist_changed(change):
    global MScurve
    MScurve.y = generateMScurve(MSdist_control.value)

def update_MS(change):
    global MScurve, MSline, MSscat
    # update line on screen
    MSline.y = MSscat.y
    
    # Update displayed MS curve on brightness diagram
    MScurve.y = generateMScurve(MSdist_control.value)

# Define function to select the star cluster we are looking at
def SC_changed(change):
    global y_sc_cluster
    
    # Hide all the star clusters
    TucData.visible = False
    M45Data.visible = False
    HyadesData.visible = False
    M53Data.visible = False
    
    # Define new brightness scale for star cluster plots
    idx = (cluster_info.Name == change.new)
    y_sc_cluster.min = float(cluster_info[idx].minBright.tolist()[0])
    y_sc_cluster.max = float(cluster_info[idx].maxBright.tolist()[0])

    # Define what data to show
    if (change.new == cluster_info.loc['m45'].Name):
        M45Data.visible = True
    elif (change.new == cluster_info.loc['hyades'].Name):
        HyadesData.visible = True
    elif (change.new == cluster_info.loc['47tuc'].Name):
        TucData.visible = True
    elif (change.new == cluster_info.loc['m53'].Name):
        M53Data.visible = True
    else:  # Shouldn't happen, but default to the Pleiades
        M45Data.visible = True

        
# Create an editable main sequence line
MScolor = np.linspace(0, 1.75, 8)
MSlum   = np.ones_like(MScolor)    # Initial flat MS curve
MSscat = bq.Scatter(x=MScolor, y=MSlum, marker='circle', default_size=50, 
                  scales={'x': x_sc_color, 'y': y_sc_lum}, colors=OverlayColor,  
                  # Make it possible to move points
                  enable_move=True, restrict_y = True, continuous_update=True)
MSline = bq.Lines(x=MSscat.x, y=MSscat.y, scales={'x': x_sc_color, 'y': y_sc_lum}, colors=OverlayColor)

In [ ]:
##
## Adjust scales/axes/tooltips as necessary from the previous plot
##

# Adjust brightness scale to the star cluster
min_bright_cluster = cluster_info.loc[init_cluster].minBright
max_bright_cluster = cluster_info.loc[init_cluster].maxBright

# Define new brightness scale for star cluster plots
y_sc_cluster = bq.LogScale(min = min_bright_cluster, max=max_bright_cluster)
ax_y_cluster  = bq.Axis(label='Brightness (L_sun/pc^2)', scale=y_sc_cluster, orientation='vertical')

# moving the label perpendicular to the axis
ax_y_cluster.label_offset = '3.5em'

In [ ]:
##
## This second pair of plots will preserve as much of the interface as the first pair for consistency
##

# Design the new HR diagram using previous Scatter object
fig_lum_cluster = bq.Figure(axes=[ax_x_color, ax_y_lum], marks=[scat_lum, MSline, MSscat],
                            animation_duration = 250, title='HR Diagram of Sample Stars',
                            background_style=PlotBkgStyle, layout=PlotLayout)

# Design brightness diagram for star clusters
TucData = bq.Scatter(x=ColorIndex[tuc], y=brightness[tuc], scales={'x': x_sc_color, 'y': y_sc_cluster}, 
                     colors=hexlist_tuc, default_size=5, stroke=None, fill=True, labels=[cluster_info.loc['47tuc'].Name])
TucData.visible = False
M45Data = bq.Scatter(x=ColorIndex[pleiades], y=brightness[pleiades], scales={'x': x_sc_color, 'y': y_sc_cluster}, 
                     colors=hexlist_m45, default_size=5, stroke=None, fill=True,  labels=[cluster_info.loc['m45'].Name])
M45Data.visible = True
M53Data = bq.Scatter(x=ColorIndex[m53], y=brightness[m53], scales={'x': x_sc_color, 'y': y_sc_cluster}, 
                     colors=hexlist_m53, default_size=5, stroke=None, fill=True, labels=[cluster_info.loc['m53'].Name])
M53Data.visible = False
HyadesData = bq.Scatter(x=ColorIndex[hyades], y=brightness[hyades], scales={'x': x_sc_color, 'y': y_sc_cluster}, 
                     colors=hexlist_hyades, default_size=5, stroke=None, fill=True, labels=[cluster_info.loc['hyades'].Name])
HyadesData.visible = False

# Generate a brightness curve for MS
initial_dist = 10
brightnessMS = generateMScurve(initial_dist)
MScurve = bq.Lines(x=MScolor, y=brightnessMS, scales={'x': x_sc_color, 'y': y_sc_cluster}, colors=OverlayColor)

# Displaying the data
fig_bright_cluster = bq.Figure(axes=[ax_x_color, ax_y_cluster], marks=[TucData, M45Data, M53Data, HyadesData, MScurve],
                               legend_location='bottom-left', legend_style={'fill': 'white'},
                               title='MS Brightness versus Star Cluster Brightness',
                               background_style=PlotBkgStyle, layout=PlotLayout)

# Select the star cluster
SC_select = widgets.RadioButtons(options=list(cluster_info.Name), 
                                 value=cluster_info.loc[init_cluster].Name, description='Cluster:', disabled=False,
                                 layout=widgets.Layout(align_content='center', align_items='center', 
                                          display='flex', 
                                          flex_flow='column', height='150px', max_height='200px', 
                                          max_width='300px', min_height='100px', min_width='125px', 
                                          overflow='hidden', width='175px'))

# Add a slider to allow adjusting distance to main sequence on brightness diagram
MSdist_control = widgets.FloatLogSlider(base=10, min=1, max=4.6, step=0.01, description='Distance (pc)', 
                                     value=initial_dist, orientation='vertical',
                                     readout=True, readout_format='04d',
                                     layout=widgets.Layout(align_content='center', align_items='center', 
                                          display='flex', 
                                          flex_flow='column', height='250px', max_height='400px', 
                                          max_width='200px', min_height='150px', min_width='50px', 
                                          overflow='hidden', width='125px'))

# Observe the main sequence to see if something is going on
MSscat.observe(update_MS, names=['y'])

# Next plot the MS line scaled by distance
MSdist_control.observe(MSdist_changed, 'value')

# Select Cluster to display from widget
SC_select.observe(SC_changed, 'value')

# Display it all
sec_disp = widgets.HBox([fig_lum_cluster, fig_bright_cluster, widgets.VBox([SC_select,MSdist_control])] , 
                     layout=widgets.Layout(align_content='center', align_items='center', 
                                           display='flex', 
                                           flex_flow='row', height='500px', max_height='500px', 
                                           max_width='1000px', min_height='400px', min_width='900px', 
                                           overflow='hidden', width='1000px'))

display(sec_disp)



## Interactive Figure 3: Estimating Age using Stellar Evolution Models
 
The figure below is allows estimation of the age of star cluster by using stellar evolution models.  

- The plot on the left shows you where the stars in a star cluster (which have widely varying mass) would appear on the HR diagram as the star cluster ages.
- The plot on the right shows a brightness diagram of a real star cluster with the location of the stars in the HR diagram shown as a blue-green line.

If you first set the distance to the cluster to the distance you previously estimated in Interactive Figure 2, then you can allow the cluster to age and use this interactive to determine both the distance and age of the star cluster.

**Note**: This interactive starts very early on, before the star cluster has really left the gas cloud it formed in.  We never see star clusters only a few hundred thousand years old.

In [ ]:
##
## Define functions to control the stellar evolution model
##

# Define a function to select data by age
def generate_age_curve(age, feh):
    global init_Z, data2plot, evol_data
    
    init_selector = (evol_data['Age'] == age) & (evol_data['FeH'] == feh)
    return evol_data[init_selector]
    
# Define function to process distance change
def age_changed(change):
    global evol_data, age_lum, Age_label, age_bright, ModelDist_control, feh_value

    # Update the label for the age
    Age_label.value = age_label_str()
        
    # Update the curve plotted
    data2plot = generate_age_curve(Ages[Age_control.value], feh_value)
    
    age_lum.x = data2plot['B-V']
    age_lum.y = data2plot['Luminosity']
    age_lum.colors = data2plot['hexcolor'].tolist()
    
    # Update the brightness curve
    ModelDist_changed(ModelDist_control)
    
    return

# Set age labe variably
def age_label_str():
    global Age_control
    # Update the label for the age
    if (Ages[Age_control.value] < 1e6):
        return "{0:.0f} years".format(Ages[Age_control.value])
    elif  (Ages[Age_control.value] < 1e9):
        return"{0:.2f} Million years".format(Ages[Age_control.value]/1e6)
    else:
        return "{0:.2f} Billion years".format(Ages[Age_control.value]/1e9)
    
    return "ERROR: You shouldn't see this."
        
# Define function to select the star cluster we are looking at in third figure
def SC2_changed(change):
    global y_sc2_cluster, TucData2, M45Data2, HyadesData2, M53Data2
    # Hide all the star clusters
    TucData2.visible = False
    M45Data2.visible = False
    HyadesData2.visible = False
    M53Data2.visible = False
    
    # Adjust rightness scale for star cluster plots
    idx = (cluster_info.LongName == change.new)
    y_sc2_cluster.min = float(cluster_info[idx].minBright.tolist()[0])
    y_sc2_cluster.max = float(cluster_info[idx].maxBright.tolist()[0])

    if (change.new == cluster_info.loc['m45'].LongName):
        M45Data2.visible = True
    elif (change.new == cluster_info.loc['hyades'].LongName):
        HyadesData2.visible = True
    elif (change.new == cluster_info.loc['47tuc'].LongName):
        TucData2.visible = True
    elif (change.new == cluster_info.loc['m53'].LongName):
        M53Data2.visible = True
    else:  # Shouldn't happen, but default to the Pleiades
        M45Data2.visible = True
    
    # Reset the metallicity
    feh_setting(change.new)
    

# Define the metallicity dataset for the cluster
def feh_setting(cluster):
    global feh_value, SC2_select
    
    # Set the metallicity
    idx = (cluster_info.LongName == SC2_select.value)
    feh_value = float(cluster_info[idx].FeH.tolist()[0])
    
    # Change label
    FeH_label.text = ["[Fe/H]: {0:.2f}".format(feh_value)]
    
    # Redraw the model data if necessary
    age_changed(cluster)
        
# Define function to generate brightness curve for a given distance
def generateBrightnessCurve(dist):
    global age_lum
    return age_lum.y/(4*np.pi*dist*dist)

# Define function to deal with distance changes
def ModelDist_changed(change):
    global ModelDist_control, age_bright, ModelDist_control
    
    age_bright.x = age_lum.x
    age_bright.y = generateBrightnessCurve(ModelDist_control.value)
    return


In [ ]:
##
## This this pair of plots will preserve as much of the interface from the previous plots as possible
##

# Get the metallicities value to assume based on initial cluster
feh_value = cluster_info.loc[init_cluster].FeH

# Set initial distance
initial_dist = 100

# Labels and scales for Axes
y_sc_lumVage = bq.LogScale(min = 10**(min_loglum+3.5), max=10**(max_loglum+3.5))
ax_y_lumVage = bq.Axis(label='Luminosity (L_sun)', scale=y_sc_lumVage, orientation='vertical')
ax_y_lumVage.label_offset = '3em'

# Get the ages stored for these interpolated models
Ages = np.sort(evol_data['Age'].unique())
n_ages = len(Ages)

# Select model data settings
init_age = Ages[0]
data2plot = generate_age_curve(init_age, feh_value)

# Adjust brightness scale to the star cluster
min_bright_cluster = cluster_info.loc[init_cluster].minBright
max_bright_cluster = cluster_info.loc[init_cluster].maxBright
y_sc2_cluster = bq.LogScale(min = min_bright_cluster, max=max_bright_cluster)

# Design brightness diagram for star clusters
TucData2 = bq.Scatter(x=ColorIndex[tuc], y=brightness[tuc], scales={'x': x_sc_color, 'y': y_sc2_cluster}, 
                     colors=hexlist_tuc, default_size=5, stroke=None, fill=True, labels=[cluster_info.loc['47tuc'].Name])
TucData2.visible = False
M45Data2 = bq.Scatter(x=ColorIndex[pleiades], y=brightness[pleiades], scales={'x': x_sc_color, 'y': y_sc2_cluster}, 
                     colors=hexlist_m45, default_size=5, stroke=None, fill=True,  labels=[cluster_info.loc['m45'].Name])
M45Data2.visible = True
M53Data2 = bq.Scatter(x=ColorIndex[m53], y=brightness[m53], scales={'x': x_sc_color, 'y': y_sc2_cluster}, 
                     colors=hexlist_m53, default_size=5, stroke=None, fill=True, labels=[cluster_info.loc['m53'].Name])
M53Data2.visible = False
HyadesData2 = bq.Scatter(x=ColorIndex[hyades], y=brightness[hyades], scales={'x': x_sc_color, 'y': y_sc_cluster}, 
                     colors=hexlist_hyades, default_size=5, stroke=None, fill=True, labels=[cluster_info.loc['hyades'].Name])
HyadesData2.visible = False

# Select the star cluster
SC2_select = widgets.RadioButtons(options=cluster_info.LongName, 
                                 value=cluster_info.loc[init_cluster].LongName, description='Cluster:', disabled=False,
                                 layout=widgets.Layout(align_content='center', align_items='center',
                                                       display='flex', flex_flow='column',
                                                       overflow='hidden',
                                                       height='125px', max_height='200px', min_height='125px',
                                                       width='250px', max_width='300px',min_width='100px'))

# Define a tooltip
tt_age_lum = bq.Tooltip(fields=['name'], formats=['5.2f'], labels=['Mass'])

# Designing the HR diagram for a given age of data
age_lum = bq.Scatter(x=data2plot['B-V'], y=data2plot['Luminosity'], names=data2plot['Mass'], display_names=False,
                     scales={'x': x_sc_color, 'y': y_sc_lumVage}, tooltip=tt_age_lum,
                     marker='circle', colors=data2plot['hexcolor'].tolist(), default_size=10, stroke=None, fill=True)

# Metallicity label
FeH_label = bq.Label(x=[0.75], y=[1e6], scales={'x': x_sc_color, 'y': y_sc_lumVage},
                    text=["[Fe/H]: {0:.2f}".format(cluster_info.loc[init_cluster].FeH)],
                    default_size=15, font_weight='bolder',
                    colors=['white'], update_on_move=False)

# Design the new HR diagram using previous Scatter object
aged_cluster = bq.Figure(axes=[ax_x_color, ax_y_lumVage], marks=[age_lum, FeH_label], 
                         title='MIST Model HR Diagram',
                         background_style=PlotBkgStyle, layout=PlotLayout)

# Compute brightness curve of the model data
brightness2plot = generateBrightnessCurve(initial_dist)
age_bright = bq.Lines(x=age_lum.x, y=brightness2plot, scales={'x': x_sc_color, 'y': y_sc2_cluster}, colors=OverlayColor)

# Displaying the cluster data and the model data on the brightness curve
fig2_bright_cluster = bq.Figure(axes=[ax_x_color, ax_y_cluster], 
                                marks=[TucData2, M45Data2, M53Data2, HyadesData2, age_bright],
                               legend_location='bottom-left', legend_style={'fill': 'white'},
                               title='Star Cluster Brightness',
                               background_style=PlotBkgStyle, layout=PlotLayout)

# Add a slider to allow adjusting distance to model data on brightness diagram
ModelDist_control = widgets.FloatLogSlider(base=10, min=1, max=4.6, step=0.01, description='Distance (pc)', 
                                     value=initial_dist, orientation='vertical',
                                     readout=True, readout_format='04d',
                                     layout=widgets.Layout(align_content='center', align_items='center',
                                                           display='flex', flex_flow='column',
                                                           overflow='hidden',
                                                           height='250px', max_height='400px', min_height='150px',
                                                           width='125px', max_width='150px',min_width='50px'))

# Add a slider to allow adjusting age of main sequence on HR diagram
Age_control = widgets.IntSlider(value = 0, min=0, max=n_ages-1, description='Age of Model Stars',
                                     orientation='vertical', readout=False, readout_format='.2e',
                                     continuous_update=True,
                                     layout=widgets.Layout(align_content='center', align_items='center',
                                                           display='flex', flex_flow='column',
                                                           overflow='hidden',
                                                           height='250px', max_height='400px', min_height='150px',
                                                           width='125px', max_width='150px',min_width='50px'))

Age_label = widgets.Label(value=age_label_str(),
                         layout=widgets.Layout(align_content='center', align_items='center',
                                               display='flex', flex_flow='column',
                                               height='25px', min_height='25px', max_height='50px',
                                               width='125px', min_width='100px', max_width='200px',
                                               overflow='hidden'))

# Play animation widget
play = widgets.Play(interval = 2, value = 0, min=0, max=n_ages-1, step=2, description="Press play", 
                    disabled=False, show_repeat=False,
                   layout=widgets.Layout(height='25px', min_height='25px', max_height='50px',
                                         width='125px', min_width='100px', max_width='200px',   
                                         overflow='hidden' ))
widgets.jslink((play, 'value'), (Age_control, 'value'))

# Direct functions to be called if various widgets change
SC2_select.observe(SC2_changed, 'value')
Age_control.observe(age_changed, 'value')
ModelDist_control.observe(ModelDist_changed, 'value')

# Build control panel
Fig3Panel = widgets.VBox([SC2_select, widgets.HBox([Age_control, ModelDist_control]), Age_label, play],
                        layout=widgets.Layout(overflow='hidden'))

# Display it all
third_disp = widgets.HBox([aged_cluster, fig2_bright_cluster, Fig3Panel] , 
                     layout=widgets.Layout(align_content='center', align_items='center', 
                                           display='flex', 
                                           flex_flow='row', height='500px', max_height='500px', 
                                           max_width='1000px', min_height='400px', min_width='900px', 
                                           overflow='hidden', width='1000px'))

display(third_disp)

